## 1. contrast (대비 or 명암)

    - contrast는 물체를 식별할 수 있도록 하는 휘도(luminance) 또는 색(color)의 차이

![contrast](img/contrast.jpg)

    - low contrast vs high contrast
        
<img src="img/low_high_contrast.png" alt='low_high_contrast' width="300" height="300">